In [ ]:
import os
import time
import json
import websocket
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class Bot:
    def __init__(self):
        __options = webdriver.ChromeOptions()
        self.browser = uc.Chrome(options=__options)
        self.old_links = []
        input("Waiting for Google Login")

    def save(self):
        url = 'ws://localhost:8080'
        def on_open(ws):
            ws.send(json.dumps({"do" : "save"}))
        try:
            ws = websocket.WebSocketApp(url, on_open=on_open)
            ws.run_forever()
            ws.close()
        except:
            pass


    def start(self):
        links = []
        self.browser.get("https://youtube.com/")
        # js code for click show more channel button
        try:
            js = '''
                function a() {
                    let a = document.querySelectorAll("ytd-guide-section-renderer")[1];
                    if (void 0 !== a) {
                        let b = a.querySelector("#items > ytd-guide-collapsible-entry-renderer");
                        return null === b ? document.createElement("foo") : a.querySelector("#items > ytd-guide-collapsible-entry-renderer").querySelector("#endpoint")
                    }
                }
                if (void 0 === a())
                    for (document.querySelector("#guide-icon").click(); void 0 === a();) await new Promise(a => setTimeout(a, 500));
                a().click();
                '''
            self.browser.execute_script(js)
        except Exception as e:
            links = self.old_links

        while len(links) == 0:
            try:
                links = self.browser.find_elements(By.XPATH, '''/html/body/ytd-app/div[1]/tp-yt-app-drawer/div[2]/div/div[
                                    2]/div[2]/ytd-guide-renderer/div[1]/ytd-guide-section-renderer[
                                    2]/div/ytd-guide-collapsible-entry-renderer/div/div/ytd-guide-entry-renderer/a''')
                links = [x.get_attribute("href") + "/videos" for x in links]
                self.old_links = links
            except:
                pass
        for channel in links:
            self.scroll(channel)

    def scroll(self, url):
        js_scroll_to_bottom = '''let scrollingElement = (document.scrollingElement || document.body);scrollingElement.scrollTop = scrollingElement.scrollHeight;'''
        self.browser.get(url)
        while True:
            try:
                self.browser.find_element(
                    By.CSS_SELECTOR, 'tp-yt-paper-spinner#spinner')
                if self.browser.find_element(By.ID, 'spinner').get_attribute("active"):
                    self.browser.execute_script(js_scroll_to_bottom)
                    time.sleep(0.2)
            except:
                self.browser.execute_script(js_scroll_to_bottom)
                time.sleep(0.4)
                return True


if __name__ == "__main__":
    bot = Bot()
    while True:
        bot.start()
        bot.save()
